In [1]:
import synapseclient
import pandas as pd
import numpy as np

syn = synapseclient.Synapse()
syn.login(silent=True)

proteomics = 'syn12177499'
proteomics = syn.get(proteomics)

proteomics_df = pd.read_json(proteomics.path, orient='records')
proteomics_df.head(5)

,ensembl_gene_id,hgnc_symbol,logfc,fc,ci_l,ci_r,adj_p_val,tissue,study,model
0,ENSG00000228521,AC099552.3,1.646715,3.131199,1.219379,2.074052,2.261704e-10,CBE,MayoRNAseq,AD Diagnosis (males and females)
1,ENSG00000163221,S100A12,1.584079,2.998163,0.890001,2.278157,1.323673e-04,CBE,MayoRNAseq,AD Diagnosis (males and females)
2,ENSG00000273802,HIST1H2BG,1.373508,2.590998,0.970485,1.776531,1.590620e-08,CBE,MayoRNAseq,AD Diagnosis (males and females)
3,ENSG00000124107,SLPI,1.369576,2.583946,0.564874,2.174278,4.777964e-03,CBE,MayoRNAseq,AD Diagnosis (males and females)
4,ENSG00000168329,CX3CR1,-1.362178,0.388995,-1.871640,-0.852716,7.491453e-06,CBE,MayoRNAseq,AD Diagnosis (males and females)


We first get summary statistics about the rna dataset and select the interesting columns.  Subsequently we rename those columns to calculate the correct interquartile range:

In [4]:
proteomics_df = proteomics_df.groupby(['tissue']).agg('describe')['logfc'].reset_index()[
    ['tissue', 'min', 'max', '25%', '50%', '75%']]

proteomics_df.rename(columns={'25%': 'first_quartile', '50%': 'median', '75%': 'third_quartile'}, inplace=True)

We calculate the quartiles correctly and round them to four significant digits.  Since we don't need the interquartile range for the portal, we drop that column.

In [5]:
proteomics_df['IQR'] = proteomics_df['third_quartile'] - proteomics_df['first_quartile']
proteomics_df['min'] = proteomics_df['first_quartile'] - (1.5 * proteomics_df['IQR'])
proteomics_df['max'] = proteomics_df['third_quartile'] + (1.5 * proteomics_df['IQR'])

for col in ['min', 'max', 'median', 'first_quartile', 'third_quartile']:
    proteomics_df[col] = np.around(proteomics_df[col], 4)

proteomics_df.drop('IQR', axis=1, inplace=True)

Lastly, we add a column for the type of the tissue.  Here 'LFQ' is hardcoded, but in the future we'll support more tissues.

In [6]:
proteomics_df['type'] = 'LFQ'